In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [15]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 5
DOLLAR_THRESHOLD = "115009542m"
EVENT_NAME = 'Z-SCORE-W100-1Z'
LABEL_NAME = "TB"

START_DATE = "20240101"
END_DATE = "20241231"

RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "interm/resampled"
PROCESSED_DIR = BASE_DIR / "processed"
SIDES_DIR = BASE_DIR / "interm/sides"
META_LABELS_DIR = BASE_DIR / "processed/meta_labels"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}_processed.pkl"

In [16]:
SIDE_NAME = "EMACross_9_36"
SIDE_FILE_PATH = SIDES_DIR / f"{RESAMPLED_NAME}-{SIDE_NAME}.pkl"

In [17]:
LABELS_FILE_PATH = META_LABELS_DIR / f"{RESAMPLED_NAME}-{SIDE_NAME}-meta.pkl"

In [23]:
print(LABELS_FILE_PATH)

../data/processed/meta_labels/USDJPY-5m-20240101-20241231-EMACross_9_36-meta.pkl


In [24]:
from torch.utils.data import Dataset, DataLoader
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [25]:
df = pd.read_pickle(PROCESSED_FILE_PATH)
labels = pd.read_pickle(LABELS_FILE_PATH)

In [26]:
print(df.shape)
df.head()

(72013, 10)


,open,high,low,close,volume,spread,return,log_return,fd_return,log_fd_return
timestamp,,,,,,,,,,
2024-01-15 06:35:00,1.096440,1.096615,1.096425,1.096595,9.697500e+11,0.000027,0.000160,0.000146,0.113814,0.009814
2024-01-15 06:40:00,1.096600,1.096620,1.096510,1.096595,5.448600e+11,0.000025,0.000000,0.000000,0.113773,0.009777
2024-01-15 06:45:00,1.096600,1.096605,1.096445,1.096525,7.287000e+11,0.000025,-0.000070,-0.000064,0.113687,0.009698
2024-01-15 06:50:00,1.096530,1.096625,1.096485,1.096545,8.230700e+11,0.000027,0.000020,0.000018,0.113715,0.009724
2024-01-15 06:55:00,1.096535,1.096545,1.096340,1.096400,9.327600e+11,0.000026,-0.000145,-0.000132,0.113565,0.009587


In [27]:
print(labels.shape)
labels.head()

(2193, 5)


,ret,bin,t1,trgt,side
timestamp,,,,,
2024-01-15 06:35:00,-0.000000,0.0,2024-01-15 09:35:00,0.014592,0
2024-01-15 12:05:00,-0.000639,0.0,2024-01-15 15:05:00,0.031122,1
2024-01-15 14:05:00,0.000000,0.0,2024-01-15 17:05:00,0.038892,0
2024-01-15 15:55:00,0.000283,1.0,2024-01-15 18:55:00,0.032986,1
2024-01-15 20:30:00,-0.000000,0.0,2024-01-15 23:30:00,0.009128,0
